In [15]:
import pyperclip

def copy_generated_autoencoder(layer_dimensions: list[int], flatten: bool = False) -> None:
    incr = sorted(layer_dimensions)
    decr = sorted(layer_dimensions, reverse=True)

    flattening_row = """
            nn.Flatten(start_dim=1),""" if flatten else ""
    
    deflattening_row = """
            nn.Unflatten(dim=1, unflattened_size=(field_shape[0], field_shape[1]))""" if flatten else ""

    result = f"""class AutoEncoder(nn.Module):

    def __init__(self, field_shape: int):
        super(AutoEncoder, self).__init__()
        self.field_shape = field_shape
        
        self.encoder = nn.Sequential({flattening_row}
            nn.Linear(field_shape[0] * field_shape[1], {decr[0]}),
            nn.ReLU(),"""
    for first, second in zip(decr, decr[1:]):
        result += f"""
            nn.Linear({first}, {second}),
            nn.ReLU(),"""
    result += """
        )
        self.decoder = nn.Sequential("""
    for first, second in zip(incr, incr[1:]):
        result += f"""
            nn.Linear({first}, {second}),
            nn.ReLU(),"""
    result += f"""
            nn.Linear({decr[0]}, field_shape[0] * field_shape[1]),
            nn.ReLU(),{deflattening_row}
        )
    
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return decode
    """

    # copy to clipboard
    pyperclip.copy(result)

    print(f"Generated autoencoder (with layer dimensions: {incr}) copied to the clipboard")

In [16]:
# TODO insert in the following list the dimensions of the layers (one int for wanted layer) in any order
layer_dimensions = [2**i for i in range(3, 10)]
#layer_dimensions = [50000, 10000, 2500, 500, 10]

copy_generated_autoencoder(layer_dimensions, flatten=True)

Generated autoencoder (with layer dimensions: [8, 16, 32, 64, 128, 256, 512]) copied to the clipboard
